First, create the model. This must match the model used in the interactive training notebook.

In [2]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [7]:
model.load_state_dict(torch.load('hellooutside.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [1]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

NameError: name 'torch' is not defined

Save the optimized model using the cell below

In [2]:
torch.save(model_trt.state_dict(), 'hellooutside.pth')

NameError: name 'torch' is not defined

Load the optimized model by executing the cell below

In [3]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('hellooutside.pth'))

<All keys matched successfully>

Create the racecar class

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=100)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [7]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 0
STEERING_BIAS = 0.00

Kp = 1.7
Kd = 6
last_x = 0

car.steering_offset = 0.18
#car.steering = 0
car.throttle = 0.

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    print(output)
    x = float(output[0])
   # car.steering = - ( x * STEERING_GAIN + STEERING_BIAS )
    car.steering =  x * Kp + (x - last_x) * Kd 
    last_x = x

[ 0.3486 -0.932 ]
[ 0.377 -0.932]
[ 0.3806 -0.9473]
[ 0.3613 -0.947 ]
[ 0.3484 -0.9385]
[ 0.3596 -0.943 ]
[ 0.343  -0.9355]
[ 0.3364 -0.9424]
[ 0.3508 -0.9478]
[ 0.347 -0.955]
[ 0.355  -0.9736]
[ 0.2732 -0.662 ]
[ 0.3005 -0.6997]
[ 0.3198 -0.6665]
[ 0.2974 -0.7163]
[ 0.2993 -0.7036]
[ 0.3127 -0.6885]
[ 0.3064 -0.687 ]
[ 0.3606 -0.687 ]
[ 0.3328 -0.7075]
[ 0.3718 -0.653 ]
[ 0.368  -0.6846]
[ 0.352  -0.6953]
[ 0.3723 -0.68  ]
[ 0.3213 -0.7197]
[ 0.2932 -0.7017]
[ 0.2964 -0.7124]
[ 0.332 -0.727]
[ 0.3284 -0.6973]
[ 0.3157 -0.6924]
[ 0.312  -0.6797]
[ 0.3103 -0.6865]
[ 0.3032 -0.693 ]
[ 0.2917 -0.6826]
[ 0.307  -0.7056]
[ 0.301 -0.704]
[ 0.2786 -0.711 ]
[ 0.2795 -0.714 ]
[ 0.2458 -0.6978]
[ 0.2937 -0.7124]
[ 0.3198 -0.7173]
[ 0.3052 -0.7354]
[ 0.3152 -0.7466]
[ 0.369  -0.6973]
[ 0.3904 -0.6973]
[ 0.4404 -0.656 ]
[ 0.4934 -0.652 ]
[ 0.485 -0.633]
[ 0.502  -0.6514]
[ 0.511  -0.6255]
[ 0.5254 -0.6377]
[ 0.514 -0.607]
[ 0.502  -0.6265]
[ 0.4236 -0.6323]
[ 0.458  -0.6084]
[ 0.368 -0.607]
[ 0.38

KeyboardInterrupt: 

### NEW CODE ADDED